In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps

2024-01-10 23:19:42.556628: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 23:19:43.481528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
# 0.08254950935282762
slimel_study = optuna.load_study(
    study_name="SLIMElastic",
    storage=get_database_url(),
)

# 0.07018143152516958
slimbpr_study = optuna.load_study(
    study_name="SLIMBPR",
    storage=get_database_url(),
)

# 0.08248515814658722
p3alpha_study = optuna.load_study(
    study_name="P3Alpha",
    storage=get_database_url(),
)

# 0.082429130748692
easer_study = optuna.load_study(
    study_name="Easer",
    storage=get_database_url(),
)

# 0.06470798788105654
userknncf_study = optuna.load_study(
    study_name="UserKNNCF",
    storage=get_database_url(),
)

# 0.07684279728158312
itemknncf_study = optuna.load_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
)

# 0.08511222934213017
p3beta_study = optuna.load_study(
    study_name="P3Beta",
    storage=get_database_url(),
)

In [4]:
recommenders = [
    {
        "recommender": ItemKNNCFRecommender,
        "params": itemknncf_study.best_params,
    },
    {
        "recommender": UserKNNCFRecommender,
        "params": userknncf_study.best_params,
    },
    {
        "recommender": RP3betaRecommender,
        "params": p3beta_study.best_params,
    },
    {
        "recommender": EASE_R_Recommender,
        "params": easer_study.best_params,
    },
    {
        "recommender": P3alphaRecommender,
        "params": p3alpha_study.best_params,
    },
    {
        "recommender": SLIM_BPR_Cython,
        "params": slimbpr_study.best_params,
    },
    {
        "recommender": SLIMElasticNetRecommender,
        "params": slimel_study.best_params,
    },
]

In [5]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, data_train, recommenders):
        super(ScoresHybridRecommender, self).__init__(data_train)

        self.data_train = sps.csr_matrix(data_train)
        self.recommenders = recommenders
        
    def prefit(self):
        for rec in self.recommenders:
            rec["recommender"] = rec["recommender"](self.data_train)
            rec["recommender"].fit(**rec["params"])
        
        
    def fit(self, weights):
        self.weights = weights      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights = []
        for rec in self.recommenders:
            item_weights.append(rec["recommender"]._compute_item_score(user_id_array, items_to_compute))

        item_weights = sum([a*b for a,b in zip(item_weights, self.weights)])
        return item_weights

In [6]:
scores_hybrid_recommender = ScoresHybridRecommender(data_train+data_val, recommenders)
scores_hybrid_recommender.prefit()

Similarity column 22222 (100.0%), 9013.87 column/sec. Elapsed time 2.47 sec
Similarity column 12638 (100.0%), 8374.48 column/sec. Elapsed time 1.51 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 4022.21 column/sec. Elapsed time 5.52 sec
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 3.66 min
P3alphaRecommender: Similarity column 22222 (100.0%), 3754.80 column/sec. Elapsed time 5.92 sec
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 6303.00 MB (41.10%) of 15334.00 MB, required is 3950.54 MB. Using sparse matrix.
Processed 12638 (100.0%) in 0.62 sec. BPR loss is 7.57E-04. Sample per second: 20508
SLIM_BPR_Recommender: Epoch 1 of 490. Elapsed time 0.55 sec
Processed 12638 (100.0%) in 1.07 sec. BPR loss is 6.99E-04. Sample per second: 11847
SLIM_BPR_Recommender: Epoch 2 of 490. Elapsed time 1.00 sec
Processed 12638 (100.0%) in 0.56 sec. BPR loss is 1.14E-03. Sample per second: 22552
SLIM_BPR_Recommender: Epoc

In [14]:
scores_hybrid_study = optuna.create_study(
    study_name="ScoresHybrid2",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-10 23:58:20,391] Using an existing study with name 'ScoresHybrid2' instead of creating a new one.


In [ ]:
scores_hybrid_recommender.fit(scores_hybrid_study.best_params.values())

In [ ]:
submission2(scores_hybrid_recommender, users, usermap, itemmap, data_train+data_val)